In [2]:

import os, json, gc
import numpy as np, pandas as pd, math, yaml

from pprint import pprint
from datetime import datetime as dt

gc.collect()


16

In [3]:

loc_exp = 'C:\\Users\\karan\\Documents\\Cricket-Analytics\\IPL\\Data\\exp\\'

location_json = 'C:\\Users\\karan\\Documents\\Cricket-Analytics\\IPL\\Data\\ipl_json_2023_05_15\\'
location_yaml = "C:\\Users\\karan\\Documents\\Cricket-Analytics\\IPL\\Data\\ipl_yaml_2023_05_15\\"


In [4]:

print(loc_exp)
print()
print(location_json)
print()
print(location_yaml)
print()
print(dt.now().date())


C:\Users\karan\Documents\Cricket-Analytics\IPL\Data\exp\

C:\Users\karan\Documents\Cricket-Analytics\IPL\Data\ipl_json_2023_05_15\

C:\Users\karan\Documents\Cricket-Analytics\IPL\Data\ipl_yaml_2023_05_15\

2023-05-17


In [5]:
#### IMPORT FILE IF YAML

In [6]:

"""
# Load YAML file
print(location_yaml + '335982.yaml')

with open(location_yaml + '335982.yaml', 'r') as file:
    data = yaml.safe_load(file)

# Initialize list to hold ball-by-ball information
balls_data = []

# Traverse through the innings
for inning in data['innings']:
    for inning_name, inning_info in inning.items():
        team = inning_info['team']
        for delivery in inning_info['deliveries']:
            for delivery_num, delivery_info in delivery.items():
                delivery_info['delivery'] = delivery_num
                delivery_info['inning'] = inning_name
                delivery_info['team'] = team
                balls_data.append(delivery_info)

# Create DataFrame from the list of dictionaries
df = pd.DataFrame(balls_data)

print(df.describe().transpose())

display(df.head(25))

"""


"\n# Load YAML file\nprint(location_yaml + '335982.yaml')\n\nwith open(location_yaml + '335982.yaml', 'r') as file:\n    data = yaml.safe_load(file)\n\n# Initialize list to hold ball-by-ball information\nballs_data = []\n\n# Traverse through the innings\nfor inning in data['innings']:\n    for inning_name, inning_info in inning.items():\n        team = inning_info['team']\n        for delivery in inning_info['deliveries']:\n            for delivery_num, delivery_info in delivery.items():\n                delivery_info['delivery'] = delivery_num\n                delivery_info['inning'] = inning_name\n                delivery_info['team'] = team\n                balls_data.append(delivery_info)\n\n# Create DataFrame from the list of dictionaries\ndf = pd.DataFrame(balls_data)\n\nprint(df.describe().transpose())\n\ndisplay(df.head(25))\n\n"

In [7]:

"""
# Define a dictionary to map team names to abbreviations
team_abbreviations = {
                'Kolkata Knight Riders': 'KKR',
                'Royal Challengers Bangalore': 'RCB',
                'Punjab Kings' : 'PBKS',
                'Chennai Super Kings' : 'CSK',
                'Mumbai Indians' : 'MI',
                'Gujarat Titans' : 'GT',
                'Sunrisers Hyderabad': 'SRH',
                'Lucknow Super Giants': 'LSG',
                'Rajasthan Royals' : 'RR',
                'Delhi Capitals' : 'DC' 
                    }

# Initialize DataFrame to hold all ball-by-ball information
all_data = pd.DataFrame()

# Specify the directory where your YAML files are stored
directory = location_yaml

# Loop over all YAML files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.yaml') and int(filename[:-5]) > 1359475:  # Check if the filename (without extension) is larger than 1359475
        print(filename)
        with open(os.path.join(directory, filename)) as file:
            data = yaml.safe_load(file)

            # Initialize list to hold ball-by-ball information
            balls_data = []

            # Traverse through the innings
            for inning in data['innings']:
                for inning_name, inning_info in inning.items():
                    print(inning_info['team'])
                    team_batting = team_abbreviations[inning_info['team']]
                    team_bowling = team_abbreviations[data['info']['teams'][1] if inning_info['team'] == data['info']['teams'][0] else data['info']['teams'][0]]
                    print(f'{team_batting} vs {team_bowling}')
                    for delivery in inning_info['deliveries']:
                        for delivery_num, delivery_info in delivery.items():
                            delivery_info['delivery'] = delivery_num
                            delivery_info['inning'] = inning_name
                            delivery_info['team_batting'] = team_batting
                            delivery_info['team_bowling'] = team_bowling
                            delivery_info['city'] = data['info']['city']
                            delivery_info['date'] = data['info']['dates'][0]
                            delivery_info['competition'] = data['info']['competition']
                            delivery_info['match_type'] = data['info']['match_type']
                            delivery_info['batsman_runs'] = delivery_info['runs']['batsman']
                            delivery_info['extra_runs'] = delivery_info['runs']['extras']
                            delivery_info['wkt'] = 1 if 'wicket' in delivery_info else 0
                            delivery_info.pop('runs', None)  # Remove the 'runs' column
                            delivery_info.pop('wicket', None)  # Remove the 'wicket' column
                            delivery_info.pop('non_striker', None)  # Remove the 'non_striker' column
                            balls_data.append(delivery_info)

            # Create DataFrame from the list of dictionaries
            df = pd.DataFrame(balls_data)

            # Rename the specified columns
            df.rename(columns={
                'batsman_runs': 'tr',
                'team_batting': 'tm_bat',
                'team_bowling': 'tm_bow',
                'batsman': 'bat',
                'bowler': 'bow',
                'match_type': 'mtype',
                'extra_runs': 'extra',
                'delivery':'bnum'
            }, inplace=True)

            # Append this DataFrame to the master DataFrame
            all_data = pd.concat([all_data, df], ignore_index=True)

print(dt.now())
"""


"\n# Define a dictionary to map team names to abbreviations\nteam_abbreviations = {\n                'Kolkata Knight Riders': 'KKR',\n                'Royal Challengers Bangalore': 'RCB',\n                'Punjab Kings' : 'PBKS',\n                'Chennai Super Kings' : 'CSK',\n                'Mumbai Indians' : 'MI',\n                'Gujarat Titans' : 'GT',\n                'Sunrisers Hyderabad': 'SRH',\n                'Lucknow Super Giants': 'LSG',\n                'Rajasthan Royals' : 'RR',\n                'Delhi Capitals' : 'DC' \n                    }\n\n# Initialize DataFrame to hold all ball-by-ball information\nall_data = pd.DataFrame()\n\n# Specify the directory where your YAML files are stored\ndirectory = location_yaml\n\n# Loop over all YAML files in the directory\nfor filename in os.listdir(directory):\n    if filename.endswith('.yaml') and int(filename[:-5]) > 1359475:  # Check if the filename (without extension) is larger than 1359475\n        print(filename)\n       

In [8]:

"""
# Define a dictionary to map team names to abbreviations
team_abbreviations = {
                'Kolkata Knight Riders': 'KKR',
                'Royal Challengers Bangalore': 'RCB',
                'Punjab Kings' : 'PBKS',
                'Chennai Super Kings' : 'CSK',
                'Mumbai Indians' : 'MI',
                'Gujarat Titans' : 'GT',
                'Sunrisers Hyderabad': 'SRH',
                'Lucknow Super Giants': 'LSG',
                'Rajasthan Royals' : 'RR',
                'Delhi Capitals' : 'DC' 
                    }

# Initialize DataFrame to hold all ball-by-ball information
all_data = pd.DataFrame()

# Specify the directory where your YAML files are stored
directory = location_yaml

# Loop over all YAML files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.yaml') and int(filename[:-5]) > 1359475:  # Check if the filename (without extension) is larger than 1359475
        print()
        print(filename)
        with open(os.path.join(directory, filename)) as file:
            data = yaml.safe_load(file)

            # Initialize list to hold ball-by-ball information
            balls_data = []

            # Initialize dictionaries to track batting order for each team
            batting_order = {team_abbreviations[team]: {} for team in data['info']['teams']}
            order_counter = {team_abbreviations[team]: 1 for team in data['info']['teams']}

            # Traverse through the innings
            for inning in data['innings']:
                for inning_name, inning_info in inning.items():
                    print(inning_info['team'])
                    team_batting = team_abbreviations[inning_info['team']]
                    team_bowling = team_abbreviations[data['info']['teams'][1] if inning_info['team'] == data['info']['teams'][0] else data['info']['teams'][0]]
                    print(f'{team_batting} vs {team_bowling}')
                    for delivery in inning_info['deliveries']:
                        for delivery_num, delivery_info in delivery.items():
                            batsman = delivery_info['batsman']
                            if batsman not in batting_order[team_batting]:
                                batting_order[team_batting][batsman] = order_counter[team_batting]
                                order_counter[team_batting] += 1
                            delivery_info['bat_order'] = batting_order[team_batting][batsman]
                            delivery_info['delivery'] = delivery_num
                            delivery_info['inning'] = inning_name
                            delivery_info['team_batting'] = team_batting
                            delivery_info['team_bowling'] = team_bowling
                            delivery_info['city'] = data['info']['city']
                            delivery_info['date'] = data['info']['dates'][0]
                            delivery_info['competition'] = data['info']['competition']
                            delivery_info['match_type'] = data['info']['match_type']
                            delivery_info['batsman_runs'] = delivery_info['runs']['batsman']
                            delivery_info['extra_runs'] = delivery_info['runs']['extras']
                            delivery_info['wkt'] = 1 if 'wicket' in delivery_info else 0
                            delivery_info.pop('runs', None)  # Remove the 'runs' column
                            delivery_info.pop('wicket', None)  # Remove the 'wicket' column
                            delivery_info.pop('non_striker', None)  # Remove the 'non_striker' column
                            balls_data.append(delivery_info)

            # Create DataFrame from the list of dictionaries
            df = pd.DataFrame(balls_data)

            # Rename the specified columns
            df.rename(columns={
                'batsman_runs': 'tr',
                'team_batting': 'tm_bat',
                'team_bowling': 'tm_bow',
                'batsman': 'bat',
                'bowler': 'bow',
                'match_type': 'mtype',
                'extra_runs': 'extra',
                'delivery':'bnum'
            }, inplace=True)

            # Append this DataFrame to the master DataFrame
            all_data = pd.concat([all_data, df], ignore_index=True)

print(dt.now())

print(all_data.describe().transpose())

display(all_data.head(3))
#display(all_data[~all_data.wicket.isna()].head(3))

"""

"\n# Define a dictionary to map team names to abbreviations\nteam_abbreviations = {\n                'Kolkata Knight Riders': 'KKR',\n                'Royal Challengers Bangalore': 'RCB',\n                'Punjab Kings' : 'PBKS',\n                'Chennai Super Kings' : 'CSK',\n                'Mumbai Indians' : 'MI',\n                'Gujarat Titans' : 'GT',\n                'Sunrisers Hyderabad': 'SRH',\n                'Lucknow Super Giants': 'LSG',\n                'Rajasthan Royals' : 'RR',\n                'Delhi Capitals' : 'DC' \n                    }\n\n# Initialize DataFrame to hold all ball-by-ball information\nall_data = pd.DataFrame()\n\n# Specify the directory where your YAML files are stored\ndirectory = location_yaml\n\n# Loop over all YAML files in the directory\nfor filename in os.listdir(directory):\n    if filename.endswith('.yaml') and int(filename[:-5]) > 1359475:  # Check if the filename (without extension) is larger than 1359475\n        print()\n        print(f

In [ ]:

import yaml
import os
import pandas as pd
from datetime import datetime as dt

print(dt.now())

# Define a dictionary to map team names to abbreviations
team_abbreviations = {
    'Kolkata Knight Riders': 'KKR',
    'Royal Challengers Bangalore': 'RCB',
    'Punjab Kings': 'PBKS',
    'Chennai Super Kings': 'CSK',
    'Mumbai Indians': 'MI',
    'Gujarat Titans': 'GT',
    'Sunrisers Hyderabad': 'SRH',
    'Lucknow Super Giants': 'LSG',
    'Rajasthan Royals': 'RR',
    'Delhi Capitals': 'DC' 
}

# Initialize DataFrame to hold all ball-by-ball information
all_data = pd.DataFrame()

# Specify the directory where your YAML files are stored
directory = location_yaml

# Initialize dictionary to hold previous batting orders
previous_bat_orders = {}

# Loop over all YAML files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.yaml') and int(filename[:-5]) > 1359475:  # Check if the filename (without extension) is larger than 1359475
        print(filename)
        with open(os.path.join(directory, filename)) as file:
            data = yaml.safe_load(file)

            # Initialize list to hold ball-by-ball information
            balls_data = []

            # Initialize dictionaries to track batting order for each team
            batting_order = {team_abbreviations[team]: {} for team in data['info']['teams']}
            order_counter = {team_abbreviations[team]: 1 for team in data['info']['teams']}

            # Traverse through the innings
            for inning in data['innings']:
                for inning_name, inning_info in inning.items():
                    team_batting = team_abbreviations[inning_info['team']]
                    team_bowling = team_abbreviations[data['info']['teams'][1] if inning_info['team'] == data['info']['teams'][0] else data['info']['teams'][0]]

                    for delivery in inning_info['deliveries']:
                        for delivery_num, delivery_info in delivery.items():
                            batsman = delivery_info['batsman']
                            if batsman not in batting_order[team_batting]:
                                batting_order[team_batting][batsman] = order_counter[team_batting]
                                order_counter[team_batting] += 1
                            delivery_info['bat_order'] = batting_order[team_batting][batsman]
                            delivery_info['delivery'] = delivery_num
                            delivery_info['inning'] = inning_name
                            delivery_info['team_batting'] = team_batting
                            delivery_info['team_bowling'] = team_bowling
                            delivery_info['city'] = data['info']['city']
                            delivery_info['date'] = data['info']['dates'][0]
                            delivery_info['competition'] = data['info']['competition']
                            delivery_info['match_type'] = data['info']['match_type']
                            delivery_info['batsman_runs'] = delivery_info['runs']['batsman']
                            delivery_info['extra_runs'] = delivery_info['runs']['extras']
                            delivery_info['wkt'] = 1 if 'wicket' in delivery_info else 0
                            delivery_info.pop('runs', None)  # Remove the 'runs' column
                            delivery_info.pop('wicket', None)  # Remove the 'wicket' column
                            delivery_info.pop('non_striker', None)  # Remove the 'non_striker' column
                            balls_data.append(delivery_info)

            # Update the previous batting orders dictionary
            for team in batting_order:
                for batsman, bat_order in batting_order[team].items():
                    if batsman not in previous_bat_orders:
                        previous_bat_orders[batsman] = []
                    previous_bat_orders[batsman].append(bat_order)

            # Create DataFrame from the list of dictionaries
            df = pd.DataFrame(balls_data)
            print(df.columns.values.tolist())

            # Rename the specified columns
            df.rename(columns={
                'batsman_runs': 'tr',
                'team_batting': 'tm_bat',
                'team_bowling': 'tm_bow',
                'batsman': 'bat',
                'bowler': 'bow',
                'match_type': 'mtype',
                'extra_runs': 'extra',
                'delivery':'bnum'
            }, inplace=True)
            print(df.columns.values.tolist())

            # Append this DataFrame to the master DataFrame
            all_data = pd.concat([all_data, df], ignore_index=True)
            print(all_data.columns.values.tolist())

            # Add rows for non-batting players
            for team in data['info']['players']:
                for player in data['info']['players'][team]:
                    if player not in batting_order[team_abbreviations[team]] and player in previous_bat_orders:
                        extra_row = {
                            'bat': player,
                            'tm_bat': team_abbreviations[team],
                            'tm_bow': team_abbreviations[data['info']['teams'][1] if team == data['info']['teams'][0] else data['info']['teams'][0]],
                            'city': data['info']['city'],
                            'date': data['info']['dates'][0],
                            'competition': data['info']['competition'],
                            'mtype': data['info']['match_type'],
                            'tr': 0,
                            'extra': 0,
                            'wkt': 0,
                            'bat_order': previous_bat_orders[player][-1]
                        }
                        all_data = all_data.append(extra_row, ignore_index=True)

print(dt.now())

print(all_data.describe().transpose())


In [10]:

print(dt.now())

df_list = []

# Define a dictionary to map team names to abbreviations
team_abbreviations = {
    'Kolkata Knight Riders': 'KKR',
    'Royal Challengers Bangalore': 'RCB',
    'Punjab Kings' : 'PBKS',
    'Chennai Super Kings' : 'CSK',
    'Mumbai Indians' : 'MI',
    'Gujarat Titans' : 'GT',
    'Sunrisers Hyderabad': 'SRH',
    'Lucknow Super Giants': 'LSG',
    'Rajasthan Royals' : 'RR',
    'Delhi Capitals' : 'DC' 
}

# Initialize DataFrame to hold all ball-by-ball information
all_data = pd.DataFrame()

# Specify the directory where your YAML files are stored
directory = location_yaml

# Initialize a dictionary to track previous batting orders for each team
previous_bat_orders = {}

# Loop over all YAML files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.yaml') and int(filename[:-5]) > 1359475:  # Check if the filename (without extension) is larger than 1359475
        print()
        print(filename)
        with open(os.path.join(directory, filename)) as file:
            data = yaml.safe_load(file)

            # Initialize list to hold ball-by-ball information
            balls_data = []

            # Initialize dictionaries to track batting order for each team
            batting_order = {team_abbreviations[team]: {} for team in data['info']['teams']}
            order_counter = {team_abbreviations[team]: 1 for team in data['info']['teams']}

            # Traverse through the innings
            for inning in data['innings']:
                for inning_name, inning_info in inning.items():
                    print(inning_info['team'])
                    team_batting = team_abbreviations[inning_info['team']]
                    team_bowling = team_abbreviations[data['info']['teams'][1] if inning_info['team'] == data['info']['teams'][0] else data['info']['teams'][0]]
                    print(f'{team_batting} vs {team_bowling}')
                    for delivery in inning_info['deliveries']:
                        for delivery_num, delivery_info in delivery.items():
                            batsman = delivery_info['batsman']
                            if batsman not in batting_order[team_batting]:
                                batting_order[team_batting][batsman] = order_counter[team_batting]
                                order_counter[team_batting] += 1
                            delivery_info['bat_order'] = batting_order[team_batting][batsman]
                            delivery_info['b.num'] = delivery_num
                            delivery_info['inning'] = inning_name
                            delivery_info['team_batting'] = team_batting
                            delivery_info['team_bowling'] = team_bowling
                            delivery_info['city'] = data['info']['city']
                            delivery_info['date'] = data['info']['dates'][0]
                            delivery_info['competition'] = data['info']['competition']
                            delivery_info['match_type'] = data['info']['match_type']
                            delivery_info['batsman_runs'] = delivery_info['runs']['batsman']
                            delivery_info['extras'] = delivery_info['runs']['extras']
                            delivery_info['wkt'] = 1 if 'wicket' in delivery_info else 0
                            delivery_info.pop('runs', None)  # Remove the 'runs' column
                            delivery_info.pop('wicket', None)  # Remove the 'wicket' column
                            delivery_info.pop('non_striker', None)  # Remove the 'non_striker' column
                            balls_data.append(delivery_info)
                        
            # Include players who didn't bat in the game
            for team in data['info']['teams']:
                team_abbr = team_abbreviations[team]
                for player in data['info']['players'][team]:
                    if player not in batting_order[team_abbr]:
                        if team_abbr in previous_bat_orders and player in previous_bat_orders[team_abbr]:
                            batting_order[team_abbr][player] = max(previous_bat_orders[team_abbr][player])
                        else:
                            batting_order[team_abbr][player] = order_counter[team_abbr]
                            order_counter[team_abbr] += 1
                        dummy_info = {
                            'bat_order': batting_order[team_abbr][player],
                            'b.num': 0,
                            'inning': 'N/A',
                            'team_batting': team_abbr,
                            'team_bowling': 'N/A',
                            'city': data['info']['city'],
                            'date': data['info']['dates'][0],
                            'competition': data['info']['competition'],
                            'match_type': data['info']['match_type'],
                            'batsman': player,
                            'batsman_runs': 0,
                            'extras': 0,
                            'wkt': 0
                        }
                        balls_data.append(dummy_info)

            # Create DataFrame from the list of dictionaries
            df = pd.DataFrame(balls_data)
            print(df.columns.values.tolist())

            # Rename the specified columns
            df.rename(columns={
                'batsman_runs': 'tr',
                'team_batting': 'tm_bat',
                'team_bowling': 'tm_bow',
                'batsman': 'bat',
                'bowler': 'bow',
                'match_type': 'mtype'
            }, inplace=True)
            
            #display(df)
            #last_index = df.index

            print(df.shape)
            print(all_data.shape)
            print(df.columns.values.tolist())
            print(all_data.columns.values.tolist())
            
            # Append this DataFrame to the master DataFrame
            all_data = pd.concat([all_data, df], ignore_index=True)            
            del df
            
print(dt.now())



2023-05-17 15:22:29.263972

1359476.yaml
Punjab Kings
PBKS vs KKR
Kolkata Knight Riders
KKR vs PBKS
['bowler', 'batsman', 'bat_order', 'b.num', 'inning', 'team_batting', 'team_bowling', 'city', 'date', 'competition', 'match_type', 'batsman_runs', 'extras', 'wkt', 'replacements']
(230, 15)
(0, 0)
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']
[]

1359477.yaml
Lucknow Super Giants
LSG vs DC
Delhi Capitals
DC vs LSG
['bowler', 'batsman', 'bat_order', 'b.num', 'inning', 'team_batting', 'team_bowling', 'city', 'date', 'competition', 'match_type', 'batsman_runs', 'extras', 'wkt', 'replacements']
(256, 15)
(230, 15)
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replace

Rajasthan Royals
RR vs CSK
Chennai Super Kings
CSK vs RR
['bowler', 'batsman', 'bat_order', 'b.num', 'inning', 'team_batting', 'team_bowling', 'city', 'date', 'competition', 'match_type', 'batsman_runs', 'extras', 'wkt', 'replacements']
(260, 15)
(3746, 15)
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']

1359492.yaml
Punjab Kings
PBKS vs GT
Gujarat Titans
GT vs PBKS
['bowler', 'batsman', 'bat_order', 'b.num', 'inning', 'team_batting', 'team_bowling', 'city', 'date', 'competition', 'match_type', 'batsman_runs', 'extras', 'wkt', 'replacements']
(256, 15)
(4006, 15)
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']
['bow', 'bat', 'bat_order', 'b.num', 'i

Royal Challengers Bangalore
RCB vs RR
Rajasthan Royals
RR vs RCB
['bowler', 'batsman', 'bat_order', 'b.num', 'inning', 'team_batting', 'team_bowling', 'city', 'date', 'competition', 'match_type', 'batsman_runs', 'extras', 'wkt', 'replacements']
(256, 15)
(7550, 15)
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']

1359507.yaml
Chennai Super Kings
CSK vs KKR
Kolkata Knight Riders
KKR vs CSK
['bowler', 'batsman', 'bat_order', 'b.num', 'inning', 'team_batting', 'team_bowling', 'city', 'date', 'competition', 'match_type', 'batsman_runs', 'extras', 'wkt', 'replacements']
(260, 15)
(7806, 15)
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']
['bow', 'bat', 'b

Kolkata Knight Riders
KKR vs SRH
Sunrisers Hyderabad
SRH vs KKR
['bowler', 'batsman', 'bat_order', 'b.num', 'inning', 'team_batting', 'team_bowling', 'city', 'date', 'competition', 'match_type', 'batsman_runs', 'extras', 'wkt', 'replacements']
(257, 15)
(11260, 15)
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']

1359522.yaml
Rajasthan Royals
RR vs GT
Gujarat Titans
GT vs RR
['bowler', 'batsman', 'bat_order', 'b.num', 'inning', 'team_batting', 'team_bowling', 'city', 'date', 'competition', 'match_type', 'batsman_runs', 'extras', 'wkt', 'replacements']
(208, 15)
(11517, 15)
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']
['bow', 'bat', 'bat_order', 'b

Gujarat Titans
GT vs SRH
Sunrisers Hyderabad
SRH vs GT
['bowler', 'batsman', 'bat_order', 'b.num', 'inning', 'team_batting', 'team_bowling', 'city', 'date', 'competition', 'match_type', 'batsman_runs', 'extras', 'wkt', 'replacements']
(255, 15)
(14907, 15)
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']
['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']
2023-05-17 15:22:45.680852


NameError: name 'pct_splits' is not defined

In [11]:

print(all_data.columns.values.tolist())

print(all_data.describe().transpose())

display(all_data.head(3))
#display(all_data[~all_data.wicket.isna()].head(3))


['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']
             count      mean       std  min  25%  50%   75%   max
bat_order  15162.0  3.993273  2.501966  1.0  2.0  3.0   6.0  12.0
b.num      15162.0  9.364696  5.874366  0.0  4.3  9.3  14.4  19.9
tr         15162.0  1.311700  1.707889  0.0  0.0  1.0   1.0   6.0
extras     15162.0  0.066878  0.345539  0.0  0.0  0.0   0.0   5.0
wkt        15162.0  0.050587  0.219160  0.0  0.0  0.0   0.0   1.0


,bow,bat,bat_order,b.num,inning,tm_bat,tm_bow,city,date,competition,mtype,tr,extras,wkt,replacements
0,UT Yadav,P Simran Singh,1,0.1,1st innings,PBKS,KKR,Chandigarh,2023-04-01,IPL,T20,0,0,0,NaN
1,UT Yadav,P Simran Singh,1,0.2,1st innings,PBKS,KKR,Chandigarh,2023-04-01,IPL,T20,2,0,0,NaN
2,UT Yadav,P Simran Singh,1,0.3,1st innings,PBKS,KKR,Chandigarh,2023-04-01,IPL,T20,0,0,0,NaN


In [12]:

print(all_data.columns.values.tolist())

display(all_data.head(11))


['bow', 'bat', 'bat_order', 'b.num', 'inning', 'tm_bat', 'tm_bow', 'city', 'date', 'competition', 'mtype', 'tr', 'extras', 'wkt', 'replacements']


,bow,bat,bat_order,b.num,inning,tm_bat,tm_bow,city,date,competition,mtype,tr,extras,wkt,replacements
0,UT Yadav,P Simran Singh,1,0.1,1st innings,PBKS,KKR,Chandigarh,2023-04-01,IPL,T20,0,0,0,NaN
1,UT Yadav,P Simran Singh,1,0.2,1st innings,PBKS,KKR,Chandigarh,2023-04-01,IPL,T20,2,0,0,NaN
2,UT Yadav,P Simran Singh,1,0.3,1st innings,PBKS,KKR,Chandigarh,2023-04-01,IPL,T20,0,0,0,NaN
3,UT Yadav,P Simran Singh,1,0.4,1st innings,PBKS,KKR,Chandigarh,2023-04-01,IPL,T20,6,0,0,NaN
4,UT Yadav,P Simran Singh,1,0.5,1st innings,PBKS,KKR,Chandigarh,2023-04-01,IPL,T20,0,0,0,NaN
5,UT Yadav,P Simran Singh,1,0.6,1st innings,PBKS,KKR,Chandigarh,2023-04-01,IPL,T20,1,0,0,NaN
6,TG Southee,P Simran Singh,1,1.1,1st innings,PBKS,KKR,Chandigarh,2023-04-01,IPL,T20,4,0,0,NaN
7,TG Southee,P Simran Singh,1,1.2,1st innings,PBKS,KKR,Chandigarh,2023-04-01,IPL,T20,4,0,0,NaN
8,TG Southee,P Simran Singh,1,1.3,1st innings,PBKS,KKR,Chandigarh,2023-04-01,IPL,T20,0,0,0,NaN
9,TG Southee,P Simran Singh,1,1.4,1st innings,PBKS,KKR,Chandigarh,2023-04-01,IPL,T20,0,0,0,NaN


In [14]:

bat_data_rup = all_data.groupby(['date','city','inning','tm_bat','bat','bat_order'])\
    .agg( {'tr':'sum','wkt':'sum', 'b.num':'count'})\
        .reset_index()\
            .sort_values(by = ['date','city','tm_bat','bat_order','inning'])


In [15]:

print(bat_data_rup.shape)

print(bat_data_rup.columns.values.tolist())


(1457, 9)
['date', 'city', 'inning', 'tm_bat', 'bat', 'bat_order', 'tr', 'wkt', 'b.num']


In [16]:

display(bat_data_rup.tail(50))


,date,city,inning,tm_bat,bat,bat_order,tr,wkt,b.num
1404,2023-05-14,Chennai,N/A,KKR,Harshit Rana,11,0,0,1
1403,2023-05-14,Chennai,N/A,KKR,CV Varun,12,0,0,1
1415,2023-05-14,Jaipur,1st innings,RCB,V Kohli,1,18,1,20
1410,2023-05-14,Jaipur,1st innings,RCB,F du Plessis,2,55,1,45
1411,2023-05-14,Jaipur,1st innings,RCB,GJ Maxwell,3,54,1,35
1414,2023-05-14,Jaipur,1st innings,RCB,MK Lomror,4,1,1,2
1412,2023-05-14,Jaipur,1st innings,RCB,KD Karthik,5,0,1,2
1413,2023-05-14,Jaipur,1st innings,RCB,MG Bracewell,6,9,0,9
1409,2023-05-14,Jaipur,1st innings,RCB,Anuj Rawat,7,29,0,11
1429,2023-05-14,Jaipur,N/A,RCB,Shahbaz Ahmed,8,0,0,1


In [19]:

print(loc_exp)

fname_exp_alldata= 'QC_batdatarup.xlsx'

bat_data_rup.to_excel(loc_exp + fname_exp_alldata, index = False)


C:\Users\karan\Documents\Cricket-Analytics\IPL\Data\exp\


In [ ]:
#### Add more columns to the all_data which might be of use 

In [30]:

# First, we'll need to extract the over from the 'bnum' column.
all_data['over'] = all_data['b.num'].apply(lambda x: int(x)+1)

# Now we can create the 'fpp', 'fmid', 'fdth' columns based on the 'over' column.
all_data['fpp'] = all_data['over'].apply(lambda x: 1 if x <= 6 else 0)
all_data['fmid'] = all_data['over'].apply(lambda x: 1 if 7 <= x <= 16 else 0)
all_data['fdth'] = all_data['over'].apply(lambda x: 1 if x > 16 else 0)

all_data.sort_values(['date','city','tm_bat','bat_order'], inplace = True)


In [31]:

all_data.groupby(['fpp','fmid','fdth']).agg({'over':'nunique'})


over
fpp fmid fdth      
0   0    1        4
    1    0       10
1   0    0        6

In [32]:

exp_fname = 'bbb_upto_G62_frm_yaml'
exp_ext = '.xlsx'

dropcols = ['extras','replacements']

all_data.drop(columns = dropcols).to_excel(loc_exp + exp_fname + exp_ext , index = False)


In [ ]:
#### END : Import yaml files

In [ ]:

folder_path = location

# Get a list of all files and directories in the folder
all_files_and_dirs = os.listdir(folder_path)

# Filter the list to include only files (exclude directories)
files_only = [f for f in all_files_and_dirs if os.path.isfile(os.path.join(folder_path, f))]

# Print total # of files
print(len(files_only))

# Print names of first few files 
print(files_only[:5])


In [ ]:

files_list = files_only
total_files = len(files_list)

#print(np.arange(total_files))
print(total_files)

# 997 files upto the 15th of May 2023 Match


In [ ]:

#All file numbers > 1359475 are #2023 IPL onwards
#All file numbers > 1304047 are #2022 IPL onwards


In [ ]:

# DEBUGGING

for filenum in np.arange(total_files):

    import_filename = files_only[filenum].replace('.json','')
    print(import_filename)
    
    if (import_filename != 'README.txt') :
        if ( int(import_filename) == 1359493 ):
            # Replace 'file.json' with the path to your JSON file
            json_file_path = location + '\\' + files_only[filenum]
            print(json_file_path)



            #adding a check to see if json file is empty before reading it with pd.read_json.

            if os.path.getsize(json_file_path) > 0:
                # Can't read it directly because of multiple structures inside the json file
                #df = pd.read_json(json_file_path)

                # let's try to flatten it and then import it
                with open(json_file_path, 'r', encoding='utf-8') as file:
                    json_content = json.load(file)

                # Flatten the nested JSON content
                df = pd.json_normalize(json_content)

            else:
                print("The JSON file is empty.")



In [ ]:

pprint(json_content)


In [ ]:

pprint(json_content['info']['city'])
pprint(json_content['info']['dates'][0])
pprint(json_content['info']['event']['match_number'])
pprint(json_content['info']['match_type'])
pprint(json_content['info']['season'])
pprint(json_content['info']['teams'][0])
pprint(json_content['info']['teams'][1])

print()

pprint(json_content['info'])
